### Common Imports

In [4]:
import warnings; warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np; seed = 2023; np.random.seed(seed)
import seaborn as sns
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold, StratifiedKFold
from sklearn.linear_model import LinearRegression, LassoCV, SGDClassifier, Ridge, Lasso, LogisticRegression, LogisticRegressionCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, make_scorer, RocCurveDisplay, accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve, r2_score, mean_absolute_error, roc_auc_score, auc, roc_curve, mean_absolute_percentage_error
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, FunctionTransformer, PolynomialFeatures
from scipy.stats import norm
%matplotlib inline

### Loading Dataset & Analysing It

In [12]:
# Load CSV
df = pd.read_csv('hockey.csv')

# Print the head of the DataFrame
print(df.head(5))

# Print out the shape of the DataFrame
print(df.shape)

# Print out unique number of items in a column
print(df['opposingTeam'].nunique())

# Print out the average of a column
print(df['icetime'].mean())

# Print out the number of specific items in a column
print(len(df[df.home_or_away == 'HOME']))

# Descriptive statistics
df.describe()

# Type of each column
df.dtypes

  opposingTeam home_or_away  icetime  gameScore  I_F_primaryAssists  \
0          STL         AWAY    118.0      -0.07                 0.0   
1          STL         AWAY   1087.0      -0.07                 0.0   
2          STL         AWAY    861.0      -0.07                 0.0   
3          STL         AWAY      0.0       0.00                 0.0   
4          STL         AWAY    108.0      -0.07                 0.0   

   I_F_secondaryAssists  I_F_shotAttempts  I_F_goals  I_F_rebounds  \
0                   0.0               0.0        0.0           0.0   
1                   0.0               2.0        0.0           0.0   
2                   0.0               1.0        0.0           0.0   
3                   0.0               0.0        0.0           0.0   
4                   0.0               1.0        0.0           0.0   

   I_F_reboundGoals  ...  I_F_hits  I_F_takeaways  I_F_giveaways  \
0               0.0  ...       0.0            0.0            0.0   
1               

opposingTeam                       object
home_or_away                       object
icetime                           float64
gameScore                         float64
I_F_primaryAssists                float64
I_F_secondaryAssists              float64
I_F_shotAttempts                  float64
I_F_goals                         float64
I_F_rebounds                      float64
I_F_reboundGoals                  float64
I_F_freeze                        float64
I_F_playContinuedInZone           float64
I_F_playContinuedOutsideZone      float64
I_F_savedShotsOnGoal              float64
I_F_savedUnblockedShotAttempts    float64
I_F_penalityMinutes               float64
I_F_faceOffsWon                   float64
I_F_hits                          float64
I_F_takeaways                     float64
I_F_giveaways                     float64
I_F_lowDangerGoals                float64
I_F_mediumDangerGoals             float64
I_F_highDangerGoals               float64
I_F_unblockedShotAttempts         